In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss
from collections import defaultdict, Counter
from file_storage import FileStorage
from heapq import heappush, heappop


from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.parse import urldefrag
import pickle
import tqdm

In [2]:
file_storage = FileStorage('../filtered_storage')
len(file_storage)

157154

In [9]:
def extract_links_from_html(url, html):
    parser = BeautifulSoup(html)
    return [
        urldefrag(urljoin(url, link.get('href'))).url
        for link in parser.findAll('a')
    ]


def build_neighbors(storage):
    result = {}
    for url, page in tqdm.tqdm(storage.items(), total=len(storage)):
        result[url] = [
            link for link in extract_links_from_html(url, page)
            if link in storage
        ]
    return result


def normilize_pagerank(pagerank, normalization_func=max):
    normalization = normalization_func(pagerank.values())
    return {url: rank / normalization for url, rank in pagerank.items()}


def calc_diff(rank_a, rank_b):
    return sum(
        (value_a - rank_b[key]) ** 2
        for key, value_a in rank_a.items()
    ) / len(rank_a)


def calc_pagerank(neighbors, iterations=20, delta=0.1, prev_pagerank=None, min_diff=1e-20):
    if prev_pagerank is None:
        prev_pagerank = defaultdict(lambda: 1 / len(neighbors))

    for ind in range(iterations):
        print('{} iteration, India rank: {}'.format(
            ind, prev_pagerank['https://simple.wikipedia.org/wiki/India']
        ))
        mean_pagerank = sum(prev_pagerank.values()) / len(prev_pagerank)
        pagerank = defaultdict(lambda: mean_pagerank * delta)
        for url in neighbors.keys():
            curr_neighbor_list = neighbors[url]
            for neighbor_url in curr_neighbor_list:
                pagerank[neighbor_url] += prev_pagerank[url] / len(curr_neighbor_list) * (1 - delta)
        pagerank = normilize_pagerank(pagerank)
        diff = calc_diff(pagerank, prev_pagerank)
        print('diff: ' + str(diff))
        if diff < min_diff:
            break
        prev_pagerank = pagerank
    return pagerank

In [10]:
neighbors = build_neighbors(file_storage)


  0%|          | 102/157154 [00:03<1:04:23, 40.65it/s]
1285it [00:50, 36.19it/s]54 [00:03<1:28:54, 29.44it/s]
  0%|          | 650/157154 [00:21<2:39:42, 16.33it/s]


  1%|          | 1366/157154 [00:42<1:15:20, 34.46it/s]


  1%|▏         | 2112/157154 [01:02<1:05:55, 39.19it/s]


  2%|▏         | 2800/157154 [01:23<1:28:54, 28.94it/s]


  2%|▏         | 3524/157154 [01:43<1:01:12, 41.83it/s]


...

 96%|█████████▋| 151530/157154 [1:00:25<02:22, 39.36it/s]


 97%|█████████▋| 152955/157154 [1:00:45<00:49, 85.25it/s]


 98%|█████████▊| 154424/157154 [1:01:04<00:29, 91.99it/s]


 99%|█████████▉| 155849/157154 [1:01:24<00:21, 61.54it/s]


100%|██████████| 157154/157154 [1:01:42<00:00, 42.45it/s]

In [16]:
with open('neighbors.pkl', 'wb') as f_out:
    pickle.dump(neighbors, f_out)

In [17]:
with open('neighbors.pkl', 'rb') as nei_file:
    neighbors = pickle.load(nei_file, encoding='utf-8')

In [14]:
neighbors_sets = {url: set(neighbor_list) for url, neighbor_list in neighbors.items()}

In [ ]:
pagerank = calc_pagerank(neighbors, iterations=50)

In [ ]:
def dump_pagerank(pagerank, path='pagerank_results.txt'):
    with open(path, 'w') as pagerank_file:
        for url, rank in page_rank.items():
            pagerank_file.write("{:>7.6} {}\n".format(rank, url))


def load_pagerank(path='pagerank_results.txt'):
    with open(path) as pagerank_file:
        pagerank = {}
        for line in pagerank_file:
            rank, url = line.strip().split()
            pagerank[url] = float(rank)
        return pagerank

In [ ]:
dump_pagerank(pagerank)

In [ ]:
def cheat_crawling(pagerank, neighbors, seed='https://simple.wikipedia.org/wiki/Main_Page'):
    priority_queue = []
    seen = set()
    pagerank_order = []
    heappush((pagerank[seed], seed))
    
    while priority_queue:
        rank, url = heappop(priority_queue)
        pagerank_order.append(rank)
        visited.add(url)
        for neighbor in neighbors[url]:
            if neighbor not in seen:
                heappush(priority_queue, (pagerank[neighbor], neighbor))\
                seen.add(neighbor)
    return pagerank_order

In [ ]:
def BFS_crawling(pagerank, neighbors, seed='https://simple.wikipedia.org/wiki/Main_Page'):
    queue = [seed]
    seen = set()
    pagerank_order = []
    
    while queue:
        url = queue.pop(0)
        pagerank_order.append(pagerank[url])
        for neighbor in neighbors[url]:
            if neighbor not in seen:
                queue.append(neighbor)
                seen.add(url)
    return pagerank_order

In [ ]:
def random_crawling(pagerank, neighbors, seed='https://simple.wikipedia.org/wiki/Main_Page'):
    queue = [seed]
    seen = set()
    pagerank_order = []
    
    while queue:
        rnd_ind = np.random.randint(0, len(queue))
        url = queue.pop(rand_ind)
        pagerank_order.append(pagerank[url])
        visited.add(url)
        for neighbor in neighbors[url]:
            if neighbor not in seen:
                queue.append(neighbor)
                seen.add(neighbor)
    return pagerank_order

In [ ]:
def priority_by_in_degree_crawling(pagerank, neighbors, seed='https://simple.wikipedia.org/wiki/Main_Page'):
    priority_queue = []
    seen = set()
    pagerank_order = []
    incoming_edges = defaultdict(set)
    for url, neighbors in neighbors.items():
        for neighbor in neighbors:
            incoming_edges[neighbor].add(url)
    heappush((len(incoming_edges[seed]), seed))
    
    while priority_queue:
        rank, url = heappop(priority_queue)
        pagerank_order.append(rank)
        visited.add(url)
        for neighbor in neighbors[url]:
            if neighbor not in seen:
                heappush(priority_queue, (len(incoming_edges[neighbor]), neighbor))
                seen.add(neighbor)
    return pagerank_order

In [ ]:
super_cheat = np.cumsum(sorted(pagerank.values()))

In [ ]:
cheat = np.cumsum(cheat_crawling(pagerank, neighbors_sets))

In [ ]:
bfs = np.cumsum(BFS_crawling(pagerank, neighbors_sets))

In [ ]:
random = np.cumsum(random_crawling(pagerank, neighbors_sets))

In [ ]:
priority_by_in_degree = np.cumsum(priority_by_in_degree_crawling(pagerank, neighbors_sets))

In [ ]:
x = np.arange(len(storage))
plt.plot(x, super_cheat, label='super_cheat')
plt.plot(x, cheat, label='cheat')
plt.plot(x, bfs, label='bfs')
plt.plot(x, random, label='random')
plt.plot(x, priority_by_in_degree, label='priority_by_in_degree')
plt.legend()
plt.show()